# 05 Score Accuracy

In [9]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')
# from optimize import optimize_tourney

## Read in data

In [10]:
opt_results = pd.read_csv('../data/opt_results.csv')
opt_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,1395_6,1138_6,1397_6,1400_6,1401_6,1274_6,1403_6,1276_6,1277_6,Chosen
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,0,0,0,0,0,0,0,0,0,1.0
1,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,0,0,0,0,0,0,0,0,0,1.0
2,2018,R1W8,W08,W09,1,1439,Virginia Tech,1104,Alabama,1439,...,0,0,0,0,0,0,0,0,0,1.0
3,2018,R1W4,W04,W13,1,1455,Wichita St,1267,Marshall,1455,...,0,0,0,0,0,0,0,0,0,1.0
4,2018,R3W1,R2W1,R2W4,3,1437,Villanova,1293,Murray St,1437,...,0,0,0,0,0,0,0,0,0,1.0


In [11]:
actuals = pd.read_csv('../data/comb_results.csv')
actuals.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,Score_Strong,NumOT_Strong,Score_Weak,NumOT_Weak,TeamID_Winner
0,1985,R1W1,W01,W16,1,1207,Georgetown,1250,Lehigh,68,0,43,0,1207
1,1985,R1W2,W02,W15,1,1210,Georgia Tech,1273,Mercer,65,0,58,0,1210
2,1985,R1W3,W03,W14,1,1228,Illinois,1318,Northeastern,76,0,57,0,1228
3,1985,R1W4,W04,W13,1,1260,Loyola-Chicago,1233,Iona,59,0,58,0,1260
4,1985,R1W5,W05,W12,1,1374,SMU,1330,Old Dominion,85,0,68,0,1374


## Filter down data

In [12]:
opt_results_v2 = opt_results[['Season' ,'Slot', 'TeamID_Winner']]
opt_results_v2.head()

,Season,Slot,TeamID_Winner
0,2018,R1W1,1437
1,2018,R2W1,1437
2,2018,R1W8,1439
3,2018,R1W4,1455
4,2018,R3W1,1437


In [13]:
actuals_v2 = actuals[['Season', 'Slot', 'Round', 'TeamID_Winner']]
actuals_v2.head()

,Season,Slot,Round,TeamID_Winner
0,1985,R1W1,1,1207
1,1985,R1W2,1,1210
2,1985,R1W3,1,1228
3,1985,R1W4,1,1260
4,1985,R1W5,1,1374


## Join data

In [14]:
pred_actuals = pd.merge(opt_results_v2, actuals_v2, on=['Season', 'Slot'], suffixes=['_pred', '_actual'], validate='1:1')

pred_actuals['Correct'] = 1*(pred_actuals['TeamID_Winner_pred']==pred_actuals['TeamID_Winner_actual'])
pred_actuals['Points'] = [2**(r-1)*c for r,c in zip(pred_actuals['Round'], pred_actuals['Correct'])]

pred_actuals.head()

,Season,Slot,TeamID_Winner_pred,Round,TeamID_Winner_actual,Correct,Points
0,2018,R1W1,1437,1,1437,1,1
1,2018,R2W1,1437,2,1437,1,2
2,2018,R1W8,1439,1,1104,0,0
3,2018,R1W4,1455,1,1267,0,0
4,2018,R3W1,1437,3,1437,1,4


In [15]:
print (len(pred_actuals))
print (pred_actuals['Correct'].sum())
print (pred_actuals['Correct'].sum()/len(pred_actuals))
print (pred_actuals['Points'].sum())

# empirical
# 63
# 24
# 0.38095238095238093
# 41

# GLM
# 63
# 27
# 0.42857142857142855
# 61

63
27
0.42857142857142855
61


In [8]:
pred_actuals.to_csv('../data/scores.csv', index=False)